<a href="https://colab.research.google.com/github/kulkarninad/HugginFace_Handson_AIAgent/blob/main/Agentic_RAG_Ninad_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datasets
from langchain_core.documents import Document

# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        page_content="\n".join([
            f"Name: {guest['name']}",
            f"Relation: {guest['relation']}",
            f"Description: {guest['description']}",
            f"Email: {guest['email']}"
        ]),
        metadata={"name": guest["name"]}
    )
    for guest in guest_dataset
]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/371 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.32k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
!pip install smolagents
!pip install langchain-community
!pip install datasets
!pip install rank_bm25
from smolagents import Tool
from langchain_community.retrievers import BM25Retriever

class GuestInfoRetrieverTool(Tool):
    name = "guest_info_retriever"
    description = "Retrieves detailed information about gala guests based on their name or relation."
    inputs = {
        "query": {
            "type": "string",
            "description": "The name or relation of the guest you want information about."
        }
    }
    output_type = "string"

    def __init__(self, docs):
        self.is_initialized = False
        self.retriever = BM25Retriever.from_documents(docs)

    def forward(self, query: str):
        results = self.retriever.get_relevant_documents(query)
        if results:
            return "\n\n".join([doc.page_content for doc in results[:3]])
        else:
            return "No matching guest information found."

            guest_info_tool = GuestInfoRetrieverTool(docs=docs)

In [ ]:
import os
from google.colab import userdata

os.environ["TOGETHER_API_KEY"] = userdata.get("TOGETHER_API_KEY")

In [ ]:
guest_info_tool = GuestInfoRetrieverTool(docs)

In [ ]:
from smolagents import CodeAgent, InferenceClientModel
import os

# Initialize the Hugging Face model with the API key
# Read the API key from the environment variable set in cell fCC49JTLYRbn
together_api_key = os.environ.get("TOGETHER_API_KEY")
if not together_api_key:
    raise ValueError("TOGETHER_API_KEY environment variable not set. Please run cell fCC49JTLYRbn and ensure the key is in Colab secrets.")

model = InferenceClientModel(api_key=together_api_key)

# Create Alfred, our gala agent, with the guest info tool
alfred = CodeAgent(tools=[guest_info_tool], model=model)

# Example query Alfred might receive during the gala
response = alfred.run("Tell me about our guest named 'Lady Ada Lovelace'.")

print("🎩 Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tell me about our guest named 'Lady Ada Lovelace'.                                                              │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  ada_info = guest_info_retriever(query="Lady Ada Lovelace")                                                       
  print(ada_info)                                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Name: Ada Lovelace
Relation: best friend
Description: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for 
her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work
on Charles Babbage's Analytical Engine.
Email: ada.lovelace@example.com

Name: Marie Curie
Relation: no relation
Description: Marie Curie was a groundbreaking physicist and chemist, famous for her research on radioactivity.
Email: marie.curie@example.com

Name: Dr. Nikola Tesla
Relation: old friend from university days
Description: Dr. Nikola Tesla is an old friend from your university days. He's recently patented a new wireless 
energy transmission system and would be delighted to discuss it with you. Just remember he's passionate about 
pigeons, so that might make for good small talk.
Email: nikola.tesla@gmail.com

Out: None

[Step 1: Duration 1.27 seconds| Input tokens: 2,087 | Output tokens: 56]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Lady Ada Lovelace is an esteemed mathematician and friend, renowned for her pioneering work in     
  mathematics and computing. She is celebrated as the first computer programmer due to her work on Charles         
  Babbage's Analytical Engine. Her email is ada.lovelace@example. com.")                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Lady Ada Lovelace is an esteemed mathematician and friend, renowned for her pioneering work in 
mathematics and computing. She is celebrated as the first computer programmer due to her work on Charles Babbage's 
Analytical Engine. Her email is ada.lovelace@example. com.

[Step 2: Duration 1.52 seconds| Input tokens: 4,504 | Output tokens: 156]

🎩 Alfred's Response:
Lady Ada Lovelace is an esteemed mathematician and friend, renowned for her pioneering work in mathematics and computing. She is celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine. Her email is ada.lovelace@example. com.


In [ ]:
!pip install ddgs
from smolagents import DuckDuckGoSearchTool

# Initialize the DuckDuckGo search tool
search_tool = DuckDuckGoSearchTool()

# Example usage
results = search_tool("Who's the current President of Pakistan?")
print(results)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.8 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0
## Search Results

[2024 Pakistani presidential election - Wikipedia](https://en.wikipedia.org/wiki/2024_Pakistani_presidential_election)
Presidential Elections were held in Pakistan on 9 March 2024 to select the 14th President of Pakistan , who is the country' s head of state. Asif Ali Zardari of Pakistan People' s Party elected as 14th President of Pakistan defeating Mahmood K...

[The President of the Islamic Republic of Pakistan](https://president.gov.pk/)
This Policy represent the policy of Pakistan Innovation hub with respect to retention and destruction of documents and other records, both in hard copy and ele

In [ ]:
from smolagents import Tool
import random

class WeatherInfoTool(Tool):
    name = "weather_info"
    description = "Fetches dummy weather information for a given location."
    inputs = {
        "location": {
            "type": "string",
            "description": "The location to get weather information for."
        }
    }
    output_type = "string"

    def forward(self, location: str):
        # Dummy weather data
        weather_conditions = [
            {"condition": "Rainy", "temp_c": 15},
            {"condition": "Clear", "temp_c": 25},
            {"condition": "Windy", "temp_c": 20}
        ]
        # Randomly select a weather condition
        data = random.choice(weather_conditions)
        return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

# Initialize the tool
weather_info_tool = WeatherInfoTool()

In [ ]:
weather_info_tool = WeatherInfoTool()

In [ ]:
from smolagents import Tool
from huggingface_hub import list_models

class HubStatsTool(Tool):
    name = "hub_stats"
    description = "Fetches the most downloaded model from a specific author on the Hugging Face Hub."
    inputs = {
        "author": {
            "type": "string",
            "description": "The username of the model author/organization to find models from."
        }
    }
    output_type = "string"

    def forward(self, author: str):
        try:
            # List models from the specified author, sorted by downloads
            models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

            if models:
                model = models[0]
                return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."
            else:
                return f"No models found for author {author}."
        except Exception as e:
            return f"Error fetching models for {author}: {str(e)}"

# Initialize the tool
hub_stats_tool = HubStatsTool()

# Example usage
print(hub_stats_tool("facebook")) # Example: Get the most downloaded model by Facebook

The most downloaded model by facebook is facebook/opt-125m with 9,178,226 downloads.


In [ ]:
hub_stats_tool = HubStatsTool()

In [ ]:
from smolagents import CodeAgent, InferenceClientModel
import os
from google.colab import userdata

# Read the API key directly from Colab secrets
together_api_key = userdata.get("TOGETHER_API_KEY")
if not together_api_key:
    raise ValueError("TOGETHER_API_KEY not found in Colab secrets. Please add your Together API key to Colab secrets with the name 'TOGETHER_API_KEY'.")

# Initialize the Hugging Face model with the API key
model = InferenceClientModel(api_key=together_api_key)

# Create Alfred with all the tools
# Ensure all tools (search_tool, weather_info_tool, hub_stats_tool) are defined by running their respective cells first.
alfred = CodeAgent(
    tools=[search_tool, weather_info_tool, hub_stats_tool],
    model=model
)

# Example query Alfred might receive during the gala
response = alfred.run("What is Facebook and what's their most popular model?")

print("🎩 Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is Facebook and what's their most popular model?                                                           │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  facebook_info = web_search(query="What is Facebook?")                                                            
  print(facebook_info)                                                                                             
  most_popular_model = hub_stats(author="facebook")                                                                
  print(most_popular_model)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Wikipedia Facebook - Wikipedia](https://en.wikipedia.org/wiki/Facebook)
2 days ago - It began as Facebook Chat in 2008, was revamped in 2010 and eventually became a standalone mobile app 
in August 2011, while remaining part of the user page on browsers. Complementing regular conversations, Messenger 
lets users make one-to-one and group voice and video calls.

[Wikipedia History of Facebook - Wikipedia](https://en.wikipedia.org/wiki/History_of_Facebook)
1 week ago - NTJ's member Moulavi Zahran Hashim, ... pro-ISIL Facebook account, known as "Al-Ghuraba" media. On May
2, 2019, at F8, the company announced its new vision with the tagline "the future is private". A redesign of the 
website and mobile app was introduced, dubbed as "FB5". The event also featured plans for improving groups, a 
dating platform, end-to-end encryption on its platforms, and allowing users on Messenger to communicate directly 
with WhatsApp and Instagram ...

[GCFGlobal Facebook: What is Facebook?](https://edu.gcfglobal.org/en/facebook101/what-is-facebook/1/)
Facebook allows you to send messages and post status updates to keep in touch with friends and family . You can 
also share different types of content, like photos and links. But sharing something on Facebook is a bit different 
from other types of online communication.

[Webwise Explained: What is Facebook? -](https://www.webwise.ie/parents/explained-what-is-facebook-2/)
August 9, 2018 - This means young people under the age of 16 in Ireland are not allowed to access this platform. 
Here, Webwise outlines some of the main issues which parents have concerns about: Privacy: Teens can sometimes 
forget that what is posted on Facebook is essentially a form of publishing and, unless ...

[Britannica Facebook | Overview, History, Controversies, & Facts | Britannica 
Money](https://www.britannica.com/money/Facebook)
1 week ago - Facebook, American online social media platform and social network service that is part of the company
Meta Platforms. Facebook was founded in 2004 by Mark Zuckerberg, Eduardo Saverin, Dustin Moskovitz, and Chris 
Hughes, all of whom were students at Harvard University.

[TechTarget What is Facebook?](https://www.techtarget.com/whatis/definition/Facebook)
Facebook is more than just a social networking platform; it's also a business . Facebook had its initial public 
offering on May 18, 2012, on the Nasdaq stock exchange trading under the symbol FB. The company rebranded as Meta 
on Oct. 28, 2021. As of Dec.

[Concordia University Irvine What is Facebook](https://www.cui.edu/aboutcui/social-media/what-is-facebook)
Learn how Concordia University Irvine supports departments using Facebook , including page vs. group guidelines, 
naming rules, logos, and best practices.

[Kontentino What is Facebook? | Social Media 
Dictionary](https://www.kontentino.com/social-media-urban-dictionary/facebook/)
Launched in 2004, it has evolved ... your news feed, joining a Facebook Group, or running an ad campaign, Facebook 
is a one-stop platform for both personal connections and brand growth ....

[Lifewire Facebook Explained: Origins, Why People Like It, and Key 
Features](https://www.lifewire.com/what-is-facebook-3486391)
May 15, 2024 - Facebook is a social networking website where users can post comments, share photographs, and post 
links to news or other interesting content on the web, chat live, and watch short-form video.

[PC.net Definition of Facebook](https://pc.net/glossary/facebook)
Facebook is a social networking website that was originally designed for college students, but is now open to 
anyone 13 years of age or older. Facebook users can create and customize their own profiles with photos, videos, 
and information about themselves.
The most downloaded model by facebook is facebook/opt-125m with 9,178,226 downloads.

Out: None

[Step 1: Duration 2.12 seconds| Input tokens: 2,174 | Output tokens: 90]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Facebook is a social networking platform that allows users to connect, share content, and engage   
  with others. The most popular model by Facebook on the Hugging Face Hub is facebook/opt-125m with 9, 178, 226    
  downloads.")                                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Facebook is a social networking platform that allows users to connect, share content, and engage with
others. The most popular model by Facebook on the Hugging Face Hub is facebook/opt-125m with 9, 178, 226 downloads.

[Step 2: Duration 1.78 seconds| Input tokens: 5,424 | Output tokens: 222]

🎩 Alfred's Response:
Facebook is a social networking platform that allows users to connect, share content, and engage with others. The most popular model by Facebook on the Hugging Face Hub is facebook/opt-125m with 9, 178, 226 downloads.


In [ ]:
guest_info_tool = load_guest_dataset()

NameError: name 'load_guest_dataset' is not defined

In [ ]:
# Import necessary libraries
import random
from smolagents import CodeAgent, InferenceClientModel

# Import our custom tools from their modules
# from tools import DuckDuckGoSearchTool, WeatherInfoTool, HubStatsTool
# from retriever import load_guest_dataset

# Ensure the cells defining DuckDuckGoSearchTool, WeatherInfoTool, HubStatsTool, and the docs/guest_info_tool are run before this cell.

# Example usage (assuming tools and docs/guest_info_tool are defined by previous cells)
# Note: This cell is likely intended for initializing an agent with multiple tools.
# The actual initialization and running of the agent would go here,
# using the tool instances created in previous cells (e.g., search_tool, weather_info_tool, hub_stats_tool, guest_info_tool).

# Example (assuming search_tool, weather_info_tool, and hub_stats_tool are defined):
# from google.colab import userdata
# together_api_key = userdata.get("TOGETHER_API_KEY")
# model = InferenceClientModel(api_key=together_api_key)
# alfred = CodeAgent(tools=[search_tool, weather_info_tool, hub_stats_tool], model=model)
# response = alfred.run("What is the weather in Paris?")
# print(response)

# Since this cell only has imports and comments, it doesn't perform an action after removing the imports.
# You would typically add code here to use the defined tools and potentially initialize an agent.

In [ ]:
# Initialize the Hugging Face model
model = InferenceClientModel()

# Initialize the web search tool
search_tool = DuckDuckGoSearchTool()

# Initialize the weather tool
weather_info_tool = WeatherInfoTool()

# Initialize the Hub stats tool
hub_stats_tool = HubStatsTool()

# Load the guest dataset and initialize the guest info tool
guest_info_tool = GuestInfoRetrieverTool(docs)

# Create Alfred with all the tools
alfred = CodeAgent(
    tools=[guest_info_tool, weather_info_tool, hub_stats_tool, search_tool],
    model=model,
    add_base_tools=True,  # Add any additional base tools
    planning_interval=3   # Enable planning every 3 steps
)

In [ ]:
import os
from google.colab import userdata

os.environ["TOGETHER_API_KEY"] = userdata.get("TOGETHER_API_KEY")

In [ ]:
#import os
#from google.colab import userdata

# Read the API key directly from Colab secrets
#together_api_key = userdata.get("TOGETHER_API_KEY")
#if not together_api_key:
 #   raise ValueError("TOGETHER_API_KEY not found in Colab secrets. Please add your Together API key to Colab secrets with the name 'TOGETHER_API_KEY'.")
query = "What's the weather like in Taiwan tonight due to typoon Ragasa? Will it be suitable for our fireworks display?"
response = alfred.run(query)

print("🎩 Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What's the weather like in Taiwan tonight due to typoon Ragasa? Will it be suitable for our fireworks display?  │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  weather_info = guest_info_retriever(query="weather in Taiwan tonight due to Typhoon Ragasa")                     
  print(weather_info)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Name: Ada Lovelace
Relation: best friend
Description: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for 
her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work
on Charles Babbage's Analytical Engine.
Email: ada.lovelace@example.com

Name: Dr. Nikola Tesla
Relation: old friend from university days
Description: Dr. Nikola Tesla is an old friend from your university days. He's recently patented a new wireless 
energy transmission system and would be delighted to discuss it with you. Just remember he's passionate about 
pigeons, so that might make for good small talk.
Email: nikola.tesla@gmail.com

Name: Marie Curie
Relation: no relation
Description: Marie Curie was a groundbreaking physicist and chemist, famous for her research on radioactivity.
Email: marie.curie@example.com

Out: None

[Step 1: Duration 1.41 seconds| Input tokens: 2,098 | Output tokens: 84]

...

[Step 3: Duration 2.34 seconds| Input tokens: 7,292 | Output tokens: 385]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID:
Root=1-68d52186-550ee7c70176ace9529108cc;b50c246e-56bd-4325-95cf-6fc53cc0cfd2)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 4: Duration 0.14 seconds]

AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: Root=1-68d52186-550ee7c70176ace9529108cc;b50c246e-56bd-4325-95cf-6fc53cc0cfd2)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [ ]:
query = "Tell me about 'Lady Ada Lovelace'"
response = alfred.run(query)

print("🎩 Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tell me about 'Lady Ada Lovelace'                                                                               │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID:
Root=1-68d521a3-19f4a17a72ff06c710f05a41;b7a65497-b211-44ec-a8c9-23c0c2ac3fd6)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 1: Duration 0.12 seconds]

AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: Root=1-68d521a3-19f4a17a72ff06c710f05a41;b7a65497-b211-44ec-a8c9-23c0c2ac3fd6)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [ ]:
query = "One of our guests is from Qwen. What can you tell me about their most popular model?"
response = alfred.run(query)

print("🎩 Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ One of our guests is from Qwen. What can you tell me about their most popular model?                            │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  guest_info = guest_info_retriever(query="guest from Qwen")                                                       
  print(guest_info)                                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Name: Dr. Nikola Tesla
Relation: old friend from university days
Description: Dr. Nikola Tesla is an old friend from your university days. He's recently patented a new wireless 
energy transmission system and would be delighted to discuss it with you. Just remember he's passionate about 
pigeons, so that might make for good small talk.
Email: nikola.tesla@gmail.com

Name: Marie Curie
Relation: no relation
Description: Marie Curie was a groundbreaking physicist and chemist, famous for her research on radioactivity.
Email: marie.curie@example.com

Name: Ada Lovelace
Relation: best friend
Description: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for 
her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work
on Charles Babbage's Analytical Engine.
Email: ada.lovelace@example.com

Out: None

[Step 1: Duration 1.23 seconds| Input tokens: 2,094 | Output tokens: 67]

...

[Step 3: Duration 2.92 seconds| Input tokens: 7,299 | Output tokens: 320]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID:
Root=1-68d521c4-616ca861021a0f7c2ad9c36b;4ffb6c8e-f32c-4867-bca1-6593354df549)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 4: Duration 0.13 seconds]

AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: Root=1-68d521c4-616ca861021a0f7c2ad9c36b;4ffb6c8e-f32c-4867-bca1-6593354df549)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [ ]:
query = "I need to speak with Dr. Nikola Tesla about recent advancements in wireless energy. Can you help me prepare for this conversation?"
response = alfred.run(query)

print("🎩 Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I need to speak with Dr. Nikola Tesla about recent advancements in wireless energy. Can you help me prepare for │
│ this conversation?                                                                                              │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID:
Root=1-68d521d8-0a9084040cc9123244af2e11;57c1734d-097c-4541-aebf-4f708b49253a)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 1: Duration 0.12 seconds]

AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: Root=1-68d521d8-0a9084040cc9123244af2e11;57c1734d-097c-4541-aebf-4f708b49253a)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [ ]:
# Create Alfred with conversation memory
alfred_with_memory = CodeAgent(
    tools=[guest_info_tool, weather_info_tool, hub_stats_tool, search_tool],
    model=model,
    add_base_tools=True,
    planning_interval=3
)

# First interaction
response1 = alfred_with_memory.run("Tell me about Lady Ada Lovelace.")
print("🎩 Alfred's First Response:")
print(response1)

# Second interaction (referencing the first)
response2 = alfred_with_memory.run("What projects is she currently working on?", reset=False)
print("🎩 Alfred's Second Response:")
print(response2)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tell me about Lady Ada Lovelace.                                                                                │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: Root=1-68d521fe-68496f442d23283a43b50f9d;ccf834d2-6545-49fe-8b58-de975f733312)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.